<a href="https://colab.research.google.com/github/Kunind27/Graph-Similarity-Computation/blob/main/nn2featuregraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline

In [ ]:
# Downloading and installing Pytorch Geometric
#!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
#!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
#!pip install torch-geometric
#import torch_geometric

In [ ]:
from torch import nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Neural Network Specifications : 
class neural_network(torch.nn.Module):
    def __init__(self, input_dim = 784, hidden_layer_dim = 25, output_dim = 10):
        super(neural_network, self).__init__()
        self.nn_model = nn.Sequential(nn.Linear(input_dim, hidden_layer_dim),
                                      nn.ReLU(),
                                      nn.Linear(hidden_layer_dim, output_dim),
                                      nn.Softmax())
    def forward(self, input):
        return self.nn_model(input)

model = neural_network(input_dim = 2, hidden_layer_dim = 3, output_dim = 1)
print(model)

neural_network(
  (nn_model): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
    (3): Softmax(dim=None)
  )
)


In [ ]:
# Demonstrating what the functions I have used do
print(list(model.children()),"\n")
print(list(model.nn_model.children()),"\n")

# State Dict
print(model.state_dict(),"\n")
print(model.state_dict().keys(),"\n")
print(model.state_dict().values())

[Sequential(
  (0): Linear(in_features=2, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
  (3): Softmax(dim=None)
)] 

[Linear(in_features=2, out_features=3, bias=True), ReLU(), Linear(in_features=3, out_features=1, bias=True), Softmax(dim=None)] 

OrderedDict([('nn_model.0.weight', tensor([[-0.1474,  0.0385],
        [ 0.5293, -0.2693],
        [ 0.7033,  0.0281]])), ('nn_model.0.bias', tensor([-0.3902, -0.1947,  0.1559])), ('nn_model.2.weight', tensor([[-0.1351,  0.5443,  0.4106]])), ('nn_model.2.bias', tensor([0.1212]))]) 

odict_keys(['nn_model.0.weight', 'nn_model.0.bias', 'nn_model.2.weight', 'nn_model.2.bias']) 

odict_values([tensor([[-0.1474,  0.0385],
        [ 0.5293, -0.2693],
        [ 0.7033,  0.0281]]), tensor([-0.3902, -0.1947,  0.1559]), tensor([[-0.1351,  0.5443,  0.4106]]), tensor([0.1212])])


In [ ]:
list(model.nn_model.state_dict().values())

[tensor([[-0.1474,  0.0385],
         [ 0.5293, -0.2693],
         [ 0.7033,  0.0281]]),
 tensor([-0.3902, -0.1947,  0.1559]),
 tensor([[-0.1351,  0.5443,  0.4106]]),
 tensor([0.1212])]

Assume that we have a sample $\mathbf{X}^{(l)} = \{\mathbf{x}^{(l)}_1, \ldots, \mathbf{x}^{(l)}_N\}, \mbox{where $N$ is total num of datapoints} ,\ \mathbf{x}^{(l)}_n = (x_{n1}^{(l)}, \ldots, x_{nm_l}^{(l)}) \in \mathbb{R}^{m_l}$ that is the input for layer $l$ with the dimension $m_l$. We operate with importance of the connection between  the neuron $i$ of the layer $l$ and the neuron $j$ of the layer $l+1$, which we define as: 
\begin{equation}
    s_{ij}^{(l)} = \frac{|w_{ij}^{(l)}| \overline{|x_{i}^{(l)}|}}{\sum_{k=1}^{m_l} |w^{(l)}_{kj}| \overline{|x_{k}^{(l)}|}+|b_j^{(l+1)}|},
\end{equation}

where $\overline{|x_{i}^{(l)}|} = \frac{1}{N}\sum_{n=1}^N |x_{ni}^{(l)}|$. 

The total importance in the neuron $j$ of the layer $l+1$ we define as $S^{(l)}_j = \sum_{k=1}^{m_l} |w^{(l)}_{kj}| \overline{|x_{k}^{(l)}|}+|b_j^{(l+1)}|, \ 1 \le j \le m_{l+1}$.


In [ ]:
np.random.seed(0)
X = np.random.randn(2,2)
print(X)

[[1.76405235 0.40015721]
 [0.97873798 2.2408932 ]]


In [ ]:
def compute_importances(model, X, device = torch.device("cpu"), mode = None):
    importance_scores = [] # X = m x 784
    neuron_importance_scores = []
    neuron_list = []
    names = list(model.nn_model.state_dict())    # Names of the parameters
    layers = list(model.nn_model.children())  # List of layers, e.g. Linear, ReLU, BatchNorm etc

    l = 0
    
    if mode == "ignore_first_layer" : 
        weight_key = f'{2*l}.weight'
        bias_key = f'{2*l}.bias'
        fc_weights = model.nn_model.state_dict()[weight_key] # n_l+1 x n_l
        fc_biases = model.nn_model.state_dict()[bias_key]

        # Score = Average strengths of the signal for every connection in the layer
        score = (X.abs().mean(dim=0)*fc_weights.abs()).cpu().detach().numpy().T # (m_l x m_l+1)
        total_neuron_importance = score.sum(axis=0) + fc_biases.abs().cpu().detach().numpy()
        # score = score/total_neuron_importance
        
        # Storing all the edge and Neuronal Importances
        # importance_scores.append(score)
        neuron_importance_scores.append(total_neuron_importance)

        X = layers[2*l+1](layers[2*l](X)) 
        l += 1
    
    # Computing Importance Scores for Each Layer : 2*(l+1) since linear layer is followed by an activation
    while (2*(l+1) < len(layers)):
        weight_key = f'{2*l}.weight'
        bias_key = f'{2*l}.bias'

        fc_weights = model.nn_model.state_dict()[weight_key] # n_l+1 x n_l
        fc_biases = model.nn_model.state_dict()[bias_key]

        # Score = Average strengths of the signal for every connection in the layer
        score = (X.abs().mean(dim=0)*fc_weights.abs()).cpu().detach().numpy().T # (m_l x m_l+1)
        total_neuron_importance = score.sum(axis=0) + fc_biases.abs().cpu().detach().numpy()
        # Normalize by the total neuron importance. So, the sum of importance sores for incoming connections == 1
        # score = score/total_neuron_importance
        
        # Storing all the edge and Neuronal Importances
        importance_scores.append(score)
        neuron_importance_scores.append(total_neuron_importance)

        # Storing number of Neurons in Current Layer
        neurons_curr_layer = fc_weights.shape[1]
        neuron_list.append(neurons_curr_layer)

        # Propagate signal forward: Linear, Activation
        X = layers[2*l+1](layers[2*l](X)) 
        l += 1   
    
    # For the Last Layer in the Network
    weight_key = f'{2*l}.weight'
    bias_key = f'{2*l}.bias'
    fc_weights = model.nn_model.state_dict()[weight_key] # n_l+1 x n_l
    fc_biases = model.nn_model.state_dict()[bias_key]

    score = (X.abs().mean(dim=0)*fc_weights.abs()).cpu().detach().numpy().T
    total_neuron_importance = score.sum(axis=0) + fc_biases.abs().cpu().detach().numpy()
    # score = score/total_neuron_importance # - Importance Score Normalization
    
    importance_scores.append(score)   
    neuron_importance_scores.append(total_neuron_importance)

    neuron_list.extend([fc_weights.shape[1], fc_weights.shape[0]])

    # Propagate signal forward: Linear, Activation
    X = layers[2*l+1](layers[2*l](X))
    
    return importance_scores#, neuron_importance_scores, neuron_list#, X

compute_importances(model, torch.tensor(X, dtype = torch.float32), device)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[array([[0.6965238 , 0.8336127 , 0.3208684 ],
        [0.6123091 , 0.8518192 , 0.85613644]], dtype=float32),
 array([[0.01108351],
        [0.21441086],
        [0.03187966]], dtype=float32)]

In [ ]:
list(model.nn_model.state_dict().values())

[tensor([[-0.1474,  0.0385],
         [ 0.5293, -0.2693],
         [ 0.7033,  0.0281]]),
 tensor([-0.3902, -0.1947,  0.1559]),
 tensor([[-0.1351,  0.5443,  0.4106]]),
 tensor([0.1212])]

# To get List of Neurons

In [ ]:
# Returns the Edges of the NN-Graph in the Edge List form
def list_of_neurons(model, mode = None):    
    l=0
    neuron_list = []
    #edge_list = []
    layers = list(model.nn_model.children())

    if mode == "ignore_first_layer":
        l+=1

    # Computing Importance Scores for Each Layer : 2*(l+1) since linear layer is followed by an activation
    while (2*(l+1) < len(layers)):
        weight_key = f'{2*l}.weight'
        bias_key = f'{2*l}.weight'

        fc_weights = model.nn_model.state_dict()[weight_key] # n_l+1 x n_l
        fc_biases = model.nn_model.state_dict()[bias_key]
        n_curr_layer = fc_weights.shape[1]
        neuron_list.append(n_curr_layer) 
        l+=1

    # For last layer
    weight_key = f'{2*l}.weight'
    fc_weights = model.nn_model.state_dict()[weight_key]
    neuron_list.extend([fc_weights.shape[1], fc_weights.shape[0]])
    print(neuron_list)
    #print(edge_list)

list_of_neurons(model)

[2, 3, 1]


# To make Edge List Given Neuron List

In [ ]:
def unordered_edge_list(nl):
    # nl -> neuron list
    total_neurons = 0
    edge_list = []

    # Iterating over Neuron Layers
    for layer in range(start, len(nl)-1):
        total_neurons += nl[layer]

        # Adding Unordered Edge_List for the Layer
        for i in range(total_neurons-nl[layer], total_neurons):
            for j in range(total_neurons, total_neurons + nl[layer+1]):
                edge_list.append([i,j])
    
    edge_list.extend([[j,i] for i,j in edge_list])
    
    return (np.array((edge_list)))


unordered_edge_list([2,3,1])

array([[0, 2],
       [0, 3],
       [0, 4],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 5],
       [3, 5],
       [4, 5],
       [2, 0],
       [3, 0],
       [4, 0],
       [2, 1],
       [3, 1],
       [4, 1],
       [5, 2],
       [5, 3],
       [5, 4]])

In [ ]:
def edge_list(nl):    
    # nl -> neuron list
    total_neurons = 0
    edge_list = []

    total_neurons += nl[0]
    # Adding Edge_List for First Layer
    for i in range(total_neurons-nl[0], total_neurons):
        for j in range(total_neurons, total_neurons + nl[1]):
            edge_list.append([i,j])

    for layer in range(1,len(nl)-1): # Exclude First and Last NN Layers
        total_neurons += nl[layer]

        for i in range(total_neurons-nl[layer], total_neurons):
            for j in range(total_neurons - nl[layer] - nl[layer-1], total_neurons - nl[layer]):
                edge_list.append([i,j])
            for j in range(total_neurons, total_neurons + nl[layer+1]):
                edge_list.append([i,j])

    # Adding Edge_List for Last Layer
    total_neurons += nl[-1]
    for i in range(total_neurons-nl[-1], total_neurons):
        for j in range(total_neurons - nl[-1] - nl[-2], total_neurons - nl[-1]):
                edge_list.append([i,j])
    
    return np.array(edge_list) # Since PyTorch Geometric Edge List is (2, num_edges)

edge_list([2,3,1])

array([[0, 2],
       [0, 3],
       [0, 4],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 0],
       [2, 1],
       [2, 5],
       [3, 0],
       [3, 1],
       [3, 5],
       [4, 0],
       [4, 1],
       [4, 5],
       [5, 2],
       [5, 3],
       [5, 4]])

# To get List of Weights in the NN

In [ ]:
# Returns a list of weights for each layer in the format : (m_l x m_l+1)
def list_of_weights(model, device = torch.device("cpu"), mode = None):
    weight_list = None

    if mode == "ignore_first_layer":
        weight_list = list(model.nn_model.state_dict().values())[2:]
    else:
        weight_list = list(model.nn_model.state_dict().values()) # The Tensors in this list are of dim : (m_l+1 x m_l)
    
    weight_array = list(map(lambda x : x.cpu().detach().T.ravel(), weight_list))
    weight_array.extend(weight_array)

    return torch.cat(weight_array[::2]) # (num_edges x 1) -> We exclude the biases 

list_of_weights(model, device)

tensor([-0.1474,  0.5293,  0.7033,  0.0385, -0.2693,  0.0281, -0.1351,  0.5443,
         0.4106, -0.1474,  0.5293,  0.7033,  0.0385, -0.2693,  0.0281, -0.1351,
         0.5443,  0.4106])

In [ ]:
def list_of_importances(importance_scores):
    importance_list = []
    for array in importance_scores:
        importance_list.extend(list(array.ravel()))
    
    importance_list.extend(importance_list)
    return importance_list

list_of_importances(compute_importances(model, torch.tensor(X, dtype = torch.float32), device))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[0.6965238,
 0.8336127,
 0.3208684,
 0.6123091,
 0.8518192,
 0.85613644,
 0.011083514,
 0.21441086,
 0.031879663,
 0.6965238,
 0.8336127,
 0.3208684,
 0.6123091,
 0.8518192,
 0.85613644,
 0.011083514,
 0.21441086,
 0.031879663]

In [ ]:
def sorting(edges, weights, importances):
    sorted_list = [w for e,w,i in sorted(zip(edges, weights, importances), key = lambda x: x[0][0])]
    sorted_edges = [e for e,w,i in sorted(zip(edges, weights, importances), key = lambda x: x[0][0])]
    sorted_importances = [i for e,w,i in sorted(zip(edges, weights, importances), key = lambda x: x[0][0])]
    s_l = [(e,w,i) for e,w,i in sorted(zip(edges, weights, importances), key = lambda x: (x[0][0], x[0][1]))]

    edge_attr = [[w,i] for e,w,i in sorted(zip(edges, weights, importances), key = lambda x: (x[0][0], x[0][1]))]
    #print(sorted_list)
    print(s_l)
    #print(sorted_list)
    #print(sorted_edges)
    print(edge_attr)

sorting(unordered_edge_list([2,3,1]), list_of_weights(model, device), list_of_importances(compute_importances(model, torch.tensor(X, dtype = torch.float32), device)))

[(array([0, 2]), tensor(-0.5079), 0.6965238), (array([0, 3]), tensor(0.6079), 0.8336127), (array([0, 4]), tensor(-0.2340), 0.3208684), (array([1, 2]), tensor(0.4637), 0.6123091), (array([1, 3]), tensor(0.6451), 0.8518192), (array([1, 4]), tensor(0.6483), 0.85613644), (array([2, 0]), tensor(-0.5079), 0.6965238), (array([2, 1]), tensor(0.4637), 0.6123091), (array([2, 5]), tensor(0.4010), 0.011083514), (array([3, 0]), tensor(0.6079), 0.8336127), (array([3, 1]), tensor(0.6451), 0.8518192), (array([3, 5]), tensor(0.1045), 0.21441086), (array([4, 0]), tensor(-0.2340), 0.3208684), (array([4, 1]), tensor(0.6483), 0.85613644), (array([4, 5]), tensor(0.1013), 0.031879663), (array([5, 2]), tensor(0.4010), 0.011083514), (array([5, 3]), tensor(0.1045), 0.21441086), (array([5, 4]), tensor(0.1013), 0.031879663)]
[[tensor(-0.5079), 0.6965238], [tensor(0.6079), 0.8336127], [tensor(-0.2340), 0.3208684], [tensor(0.4637), 0.6123091], [tensor(0.6451), 0.8518192], [tensor(0.6483), 0.85613644], [tensor(-0.50

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
